In [1]:
pip install requests


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# import os
# import requests
# from urllib.parse import urlencode
# from PIL import Image
# from io import BytesIO

# # === CONFIG ===
# PEXELS_API_KEY = 'tgyCOuysOP0QxuAdcAJcZNLZ72KgjQRMDmLYShTwKujsXoGp803y0SDG'
# UNSPLASH_ACCESS_KEY = 'cBqeInHXOyEHJ7lEMlOJaG7HLrxj6szTYhk84ZuUPCM'
# SAVE_DIR = 'downloaded_images'
# CATEGORIES = ['animal', 'people', 'street']
# IMAGES_PER_CATEGORY = 1500  # Adjust as needed

# # === UTILS ===
# def save_image_from_url(url, save_path):
#     try:
#         response = requests.get(url, timeout=10)
#         image = Image.open(BytesIO(response.content))
#         image.save(save_path)
#         return True
#     except Exception as e:
#         print(f"Failed to save image from {url}: {e}")
#         return False

# def ensure_dir(path):
#     os.makedirs(path, exist_ok=True)

# # === PEXELS SCRAPER ===
# def scrape_from_pexels(query, max_images, save_folder):
#     headers = {'Authorization': PEXELS_API_KEY}
#     per_page = 30
#     total_downloaded = 0
#     page = 1

#     while total_downloaded < max_images:
#         params = {'query': query, 'per_page': per_page, 'page': page}
#         response = requests.get('https://api.pexels.com/v1/search', headers=headers, params=params)
#         data = response.json()
#         photos = data.get('photos', [])
#         if not photos:
#             break

#         for photo in photos:
#             url = photo['src']['large']
#             save_path = os.path.join(save_folder, f"pexels_{total_downloaded}.jpg")
#             if save_image_from_url(url, save_path):
#                 total_downloaded += 1
#             if total_downloaded >= max_images:
#                 break
#         page += 1

# # === UNSPLASH SCRAPER ===
# def scrape_from_unsplash(query, max_images, save_folder):
#     per_page = 30
#     total_downloaded = 0
#     page = 1

#     while total_downloaded < max_images:
#         params = urlencode({
#             'query': query,
#             'page': page,
#             'per_page': per_page,
#             'client_id': UNSPLASH_ACCESS_KEY
#         })
#         url = f"https://api.unsplash.com/search/photos?{params}"
#         response = requests.get(url)
#         data = response.json()
#         results = data.get('results', [])
#         if not results:
#             break

#         for result in results:
#             url = result['urls']['regular']
#             save_path = os.path.join(save_folder, f"unsplash_{total_downloaded}.jpg")
#             if save_image_from_url(url, save_path):
#                 total_downloaded += 1
#             if total_downloaded >= max_images:
#                 break
#         page += 1

# # === MAIN SCRIPT ===
# def main():
#     for category in CATEGORIES:
#         save_path = os.path.join(SAVE_DIR, category)
#         ensure_dir(save_path)
#         print(f"\n--- Downloading category: {category} ---")

#         print("From Pexels...")
#         scrape_from_pexels(category, IMAGES_PER_CATEGORY // 2, save_path)

#         print("From Unsplash...")
#         scrape_from_unsplash(category, IMAGES_PER_CATEGORY // 2, save_path)

# if __name__ == '__main__':
#     main()

import os
import requests
from urllib.parse import urlencode
from PIL import Image
from io import BytesIO

# === CONFIG ===
PEXELS_API_KEY = 'tgyCOuysOP0QxuAdcAJcZNLZ72KgjQRMDmLYShTwKujsXoGp803y0SDG'
UNSPLASH_ACCESS_KEY = 'cBqeInHXOyEHJ7lEMlOJaG7HLrxj6szTYhk84ZuUPCM'
SAVE_DIR = 'downloaded_images'
CATEGORIES = ['animal', 'people', 'street']
IMAGES_PER_CATEGORY = 1500

# === UTILS ===
def save_image_from_url(url, save_path):
    try:
        response = requests.get(url, timeout=10)
        image = Image.open(BytesIO(response.content))
        image.save(save_path)
        return True
    except Exception as e:
        print(f"❌ Failed to save image from {url}: {e}")
        return False

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def count_images_by_prefix(folder, prefix):
    return len([f for f in os.listdir(folder) if f.startswith(prefix) and f.lower().endswith('.jpg')])

# === PEXELS SCRAPER ===
def scrape_from_pexels(query, max_images, save_folder, start_index=0):
    headers = {'Authorization': PEXELS_API_KEY}
    per_page = 30
    total_downloaded = 0
    page = 1

    while total_downloaded + start_index < max_images:
        params = {'query': query, 'per_page': per_page, 'page': page}
        try:
            response = requests.get('https://api.pexels.com/v1/search', headers=headers, params=params)
            data = response.json()
        except Exception as e:
            print(f"❌ Pexels error: {e}")
            break

        photos = data.get('photos', [])
        if not photos:
            break

        for photo in photos:
            index = start_index + total_downloaded
            save_path = os.path.join(save_folder, f"pexels_{index}.jpg")
            if save_image_from_url(photo['src']['large'], save_path):
                total_downloaded += 1
            if start_index + total_downloaded >= max_images:
                break
        page += 1

    print(f"✅ Pexels: Downloaded {total_downloaded} new images.")

# === UNSPLASH SCRAPER ===
def scrape_from_unsplash(query, max_images, save_folder, start_index=0):
    per_page = 30
    total_downloaded = 0
    page = 1

    while total_downloaded + start_index < max_images:
        try:
            params = urlencode({
                'query': query,
                'page': page,
                'per_page': per_page,
                'client_id': UNSPLASH_ACCESS_KEY
            })
            url = f"https://api.unsplash.com/search/photos?{params}"
            response = requests.get(url)
            data = response.json()
            results = data.get('results', [])
            if not results:
                break
        except Exception as e:
            print(f"❌ Unsplash error: {e}")
            break

        for result in results:
            index = start_index + total_downloaded
            save_path = os.path.join(save_folder, f"unsplash_{index}.jpg")
            if save_image_from_url(result['urls']['regular'], save_path):
                total_downloaded += 1
            if start_index + total_downloaded >= max_images:
                break
        page += 1

    print(f"✅ Unsplash: Downloaded {total_downloaded} new images.")
    return total_downloaded

# === MAIN ===
def main():
    for category in CATEGORIES:
        save_path = os.path.join(SAVE_DIR, category)
        ensure_dir(save_path)

        print(f"\n📂 Category: {category}")
        unsplash_existing = count_images_by_prefix(save_path, 'unsplash_')
        pexels_existing = count_images_by_prefix(save_path, 'pexels_')

        unsplash_target = IMAGES_PER_CATEGORY // 2
        pexels_target = IMAGES_PER_CATEGORY - unsplash_target

        print(f"▶️ Resuming Unsplash (have {unsplash_existing}/{unsplash_target})")
        unsplash_downloaded = scrape_from_unsplash(category, unsplash_target, save_path, start_index=unsplash_existing)

        # If Unsplash failed or didn't complete
        pexels_needed = IMAGES_PER_CATEGORY - (unsplash_existing + unsplash_downloaded + pexels_existing)
        if pexels_needed > 0:
            print(f"▶️ Resuming Pexels (need {pexels_needed})")
            scrape_from_pexels(category, pexels_existing + pexels_needed, save_path, start_index=pexels_existing)

if __name__ == '__main__':
    main()





📂 Category: animal
▶️ Resuming Unsplash (have 750/750)
✅ Unsplash: Downloaded 0 new images.

📂 Category: people
▶️ Resuming Unsplash (have 0/750)
❌ Failed to save image from https://images.unsplash.com/photo-1607748851687-ba9a10438621?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3NDU1OTJ8MHwxfHNlYXJjaHwzMDR8fHBlb3BsZXxlbnwwfHx8fDE3NDYxMTYyODh8MA&ixlib=rb-4.0.3&q=80&w=1080: HTTPSConnectionPool(host='images.unsplash.com', port=443): Max retries exceeded with url: /photo-1607748851687-ba9a10438621?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3NDU1OTJ8MHwxfHNlYXJjaHwzMDR8fHBlb3BsZXxlbnwwfHx8fDE3NDYxMTYyODh8MA&ixlib=rb-4.0.3&q=80&w=1080 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc77d3e6600>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
✅ Unsplash: Downloaded 750 new images.

📂 Category: street
▶️ Resuming Unsplash (have 0/750)
❌ Failed to save image from https://images.unsplash.com/photo-1517776832751-0a7e6993de03?c

In [1]:
import os
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# === Load BLIP model and processor ===
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

# === Directory paths ===
IMAGE_ROOT = "downloaded_images"
OUTPUT_FILE = "blip_captions.txt"

# === Caption generation ===
def generate_caption(img_path):
    try:
        image = Image.open(img_path).convert("RGB")
        inputs = processor(image, return_tensors="pt").to("cuda")
        output = model.generate(**inputs, max_new_tokens=30)
        return processor.decode(output[0], skip_special_tokens=True)
    except Exception as e:
        print(f"⚠️ Error generating caption for {img_path}: {e}")
        return None

# === Main script ===
def main():
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        for root, _, files in os.walk(IMAGE_ROOT):
            for file in files:
                if file.lower().endswith((".jpg", ".jpeg", ".png")):
                    img_path = os.path.join(root, file)
                    print(f"🖼️ Processing: {img_path}")
                    caption = generate_caption(img_path)
                    if caption:
                        rel_path = os.path.relpath(img_path, IMAGE_ROOT)
                        f.write(f"{rel_path}\t{caption}\n")

    print(f"\n✅ Captions saved to: {OUTPUT_FILE}")

if __name__ == "__main__":
    main()


/home/vitoupro/code/image_captioning/image_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🖼️ Processing: downloaded_images/people/unsplash_181.jpg
🖼️ Processing: downloaded_images/people/unsplash_43.jpg
🖼️ Processing: downloaded_images/people/unsplash_190.jpg
🖼️ Processing: downloaded_images/people/pexels_200.jpg
🖼️ Processing: downloaded_images/people/unsplash_358.jpg
🖼️ Processing: downloaded_images/people/unsplash_436.jpg
🖼️ Processing: downloaded_images/people/unsplash_39.jpg
🖼️ Processing: downloaded_images/people/unsplash_387.jpg
🖼️ Processing: downloaded_images/people/unsplash_14.jpg
🖼️ Processing: downloaded_images/people/pexels_441.jpg
🖼️ Processing: downloaded_images/people/unsplash_659.jpg
🖼️ Processing: downloaded_images/people/unsplash_42.jpg
🖼️ Processing: downloaded_images/people/unsplash_610.jpg
🖼️ Processing: downloaded_images/people/pexels_373.jpg
🖼️ Processing: downloaded_images/people/unsplash_199.jpg
🖼️ Processing: downloaded_images/people/pexels_621.jpg
🖼️ Processing: downloaded_images/people/pexels_326.jpg
🖼️ Processing: downloaded_images/people/unspl

In [2]:
pip install deep-translator


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from deep_translator import GoogleTranslator
import os

# === File paths ===
input_file = "blip_captions.txt"
output_file = "blip_captions_khmer.txt"

# === Setup translator ===
translator = GoogleTranslator(source='en', target='km')

# === Translate file ===
def translate_file():
    if not os.path.exists(input_file):
        print(f"❌ File not found: {input_file}")
        return

    with open(input_file, "r", encoding="utf-8") as fin, \
         open(output_file, "w", encoding="utf-8") as fout:
        
        for line in fin:
            try:
                path, caption = line.strip().split("\t", 1)
                khmer_caption = translator.translate(text=caption)
                fout.write(f"{path}\t{khmer_caption}\n")
            except Exception as e:
                print(f"⚠️ Error translating line: {line.strip()} — {e}")
    
    print(f"\n✅ Khmer captions saved to: {output_file}")

if __name__ == "__main__":
    translate_file()


⚠️ Error translating line: animal/pexels_84.jpg	a black horse running through a field of flowers — a black horse running through a field of flowers --> No translation was found using the current translator. Try another translator?

✅ Khmer captions saved to: blip_captions_khmer.txt


In [1]:
import os
import re

# Paths
input_file = "blip_captions_khmer.txt"
output_file = "blip_captions_khmer_sorted.txt"

# Define expected image order
ordered_image_paths = []

# Animal
ordered_image_paths += [f"animal/pexels_{i}.jpg" for i in range(968)]
ordered_image_paths += [f"animal/unsplash_{i}.jpg" for i in range(750)]

# People
ordered_image_paths += [f"people/pexels_{i}.jpg" for i in range(753)]
ordered_image_paths += [f"people/unsplash_{i}.jpg" for i in range(750)]

# Street
ordered_image_paths += [f"street/pexels_{i}.jpg" for i in range(753)]
ordered_image_paths += [f"street/unsplash_{i}.jpg" for i in range(750)]

# Load caption lines into dictionary with auto-separator detection
caption_dict = {}
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        # Try to split using tab, fallback to whitespace split
        if "\t" in line:
            path, caption = line.split("\t", 1)
        else:
            match = re.match(r"^(\S+)\s+(.*)$", line)
            if not match:
                print(f"⚠️ Skipping malformed line: {line}")
                continue
            path, caption = match.groups()
        caption_dict[path.strip()] = caption.strip()

# Write in correct order
missing = 0
with open(output_file, "w", encoding="utf-8") as f:
    for path in ordered_image_paths:
        if path in caption_dict:
            f.write(f"{path}\t{caption_dict[path]}\n")
        else:
            missing += 1
            print(f"⚠️ Missing caption for: {path}")

print(f"\n✅ Sorted captions written to: {output_file}")
if missing:
    print(f"⚠️ Total missing captions: {missing}")



✅ Sorted captions written to: blip_captions_khmer_sorted.txt


In [2]:
import re

file_path = "blip_captions_khmer_sorted.txt"

# Read and fix lines
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

fixed_lines = []
for line in lines:
    line = line.strip()
    if not line:
        continue
    match = re.match(r"^(\S+)\s+(.*)$", line)
    if match:
        image_path, caption = match.groups()
        fixed_lines.append(f"{image_path} {caption.strip()}\n")
    else:
        print(f"⚠️ Skipping malformed line: {line}")

# Overwrite the original file with cleaned content
with open(file_path, "w", encoding="utf-8") as f:
    f.writelines(fixed_lines)

print(f"✅ Cleaned and updated: {file_path}")


✅ Cleaned and updated: blip_captions_khmer_sorted.txt
